In [ ]:
!unzip '../input/carvana-image-masking-challenge/train.zip'
!unzip '../input/carvana-image-masking-challenge/train_masks.zip'

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
import numpy as np
import pandas as pd 
from IPython.display import clear_output
import matplotlib.pyplot as plt
import glob
from PIL import Image
from keras.preprocessing import image
import cv2


In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
def preprocess_image(img,mask,train=True):
    input_img=cv2.resize(img,(128,128))/255.0
    input_mask=cv2.resize(mask,(128,128))
    return input_img,input_mask

In [ ]:
def load_imgs(name):
    input_img=cv2.imread('./train/'+name+'.jpg')
    input_mask=image.img_to_array(Image.open('./train_masks/'+name+'_mask.gif'))
    input_img,input_mask=preprocess_image(input_img,input_mask)
    return input_img,input_mask

In [ ]:
x_data=[]
y_data=[]
imgs_path=glob.glob('./train/*')
for i in range(len(imgs_path)):
    input_img,input_mask=load_imgs(imgs_path[i][8:-4])
    x_data.append(input_img)
    y_data.append(input_mask)

In [ ]:
x_d=np.array(x_data)
y_d=np.array(y_data)
train_data=int((x_d.shape[0]*0.80))
x_train=x_d[:train_data]
y_train=y_d[:train_data]
x_test=x_d[train_data:]
y_test=y_d[train_data:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

y_train=y_train[...,np.newaxis]
y_test=y_test[...,np.newaxis]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

## Definir modelo

Definimos dos salidas a diferenciar, el fondo y el carro, generando una mascara del carro.

In [ ]:

OUTPUT_CHANNELS = 2

Para definir la red hacemos uso de MobileNetV2, entendimos que este es un modelo que hace uso de la luz presente en imagen para obtener la mayor cantidad de información, la entrada corresponde a las imagenes redimensionandas y rgb.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

Por medio de upstack se van a concatenar capas a la red, que van a redimensionar la imagen que luego de ser reducidas para obtener la informacion relevante. Pix2Pix es capaz de reconstruir la estructura de la imagen a partir de una dimension menor. (Honestamente no entendimos bien como saber cuantos filtros aplicar y el size del kernel, es decir el 512 y el 3)

In [ ]:
up_stack = [
    #pix2pix.upsample(1024, 3),
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]


Al darle mayor número de filtros y tamaño de kernel, el tiempo de entrenamiento de la red, aumenta bastante.

In [ ]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 4, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

El modelo en general, extrae las características reduciendo las dimensiones, por lo que la construccion de la mascara se enfoca en volver a dimensionar la imagen?

## Entrena el modelo

In [ ]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model_history = model.fit(x_train,y_train, epochs=1,validation_data=(x_test,y_test))

El modelo entrenado resultaba obtener unas mascaras bastante similares a las de test, el numero de epocas de entrenamiento lo variamos pero el tiempo de proceso era bastante mayor y aunque la precision aumentaba, nos pareció que llegaba a un punto donde se sobreentrenaba la red. 

In [ ]:
i=4
pred_mask = model.predict(x_test[i:i+1])
print(pred_mask.shape)
y_pred=np.argmax(pred_mask[0],-1)
y_pred=y_pred[...,np.newaxis]
print(y_pred.shape)
fig=plt.figure(figsize=(15, 15))
img=x_test[i]
true_mask=keras.preprocessing.image.array_to_img(y_test[i])
pred_mask=keras.preprocessing.image.array_to_img(y_pred)
files=[img,true_mask,pred_mask]
for i in range(len(files)):
    plt.subplot(1, len(files) , i+1)
    plt.imshow((files[i]))